# Importing Libraries

In [1]:
from lxml import etree
import numpy as np
import pandas as pd
import os
from skimage import io
from skimage.transform import resize

import keras
from keras.layers import Input, InputLayer, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, MaxPool2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import scipy.misc
from PIL import Image
from keras.utils import np_utils

os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 

import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential


Using TensorFlow backend.


# Data Loading

In [2]:
# parameters that you should set before running this script
# select class, this default should yield 1489 training and 1470 validation images

filter = ['dog', 'bird', 'boat']

# image size that you will use for your network (input images will be resampled to this size), 
image_size = 192    # 256

n_classes=3
img_x, img_y = image_size, image_size
n_channels = 3 # RGB

# please replace with the location on your laptop where you unpacked the tarball  
voc_root_folder = "C:/Users/Sumi/Desktop/NOTES SEM2/Computer Vision/CV Project/VOCdevkit"


# step1 - build list of filtered filenames
annotation_folder = os.path.join(voc_root_folder, "VOC2009/Annotations/")
annotation_files = os.listdir(annotation_folder)
filtered_filenames = []
for a_f in annotation_files:
    tree = etree.parse(os.path.join(annotation_folder, a_f))
    if np.any([tag.text == filt for tag in tree.iterfind(".//name") for filt in filter]):
        filtered_filenames.append(a_f[:-4])

# step2 - build (x,y) for TRAIN/VAL (classification)
classes_folder = os.path.join(voc_root_folder, "VOC2009/ImageSets/Main/")
classes_files = os.listdir(classes_folder)
train_files = [os.path.join(classes_folder, c_f) for filt in filter for c_f in classes_files if filt in c_f and '_train.txt' in c_f]
val_files   = [os.path.join(classes_folder, c_f) for filt in filter for c_f in classes_files if filt in c_f and '_val.txt' in c_f]


def build_classification_dataset(list_of_files):
    """ build training or validation set

    :param list_of_files: list of filenames to build trainset with
    :return: tuple with x np.ndarray of shape (n_images, image_size, image_size, 3) and  y np.ndarray of shape (n_images, n_classes)
    """
    temp = []
    train_labels = []
    for f_cf in list_of_files:
        with open(f_cf) as file:
            lines = file.read().splitlines()
            temp.append([line.split()[0] for line in lines if int(line.split()[-1]) == 1])
            label_id = [f_ind for f_ind, filt in enumerate(filter) if filt in f_cf][0]
            train_labels.append(len(temp[-1]) * [label_id])
    train_filter = [item for l in temp for item in l]

    image_folder = os.path.join(voc_root_folder, "VOC2009/JPEGImages/")
    image_filenames = [os.path.join(image_folder, file) for f in train_filter for file in os.listdir(image_folder) if
                       f in file]
    x = np.array([resize(io.imread(img_f), (image_size, image_size, 3)) for img_f in image_filenames]).astype(
        'float32')
    # changed y to an array of shape (num_examples, num_classes) with 0 if class is not present and 1 if class is present
    y_temp = []
    for tf in train_filter:
        y_temp.append([1 if tf in l else 0 for l in temp])
    y = np.array(y_temp)

    return x, y

# Images are in RGB format

x_train, y_train = build_classification_dataset(train_files)
print('%i training images from %i classes' %(x_train.shape[0], y_train.shape[1]))
x_val, y_val = build_classification_dataset(val_files)
print('%i validation images from %i classes' %(x_val.shape[0],  y_train.shape[1]))
print("x:", x_train.shape, x_val.shape)
print("y:", y_train.shape, y_val.shape)


748 training images from 3 classes
731 validation images from 3 classes
x: (748, 192, 192, 3) (731, 192, 192, 3)
y: (748, 3) (731, 3)


## Loading Data into Pkl files to use in Google colab

In [8]:
X_Train = x_train.copy()
X_Val = x_val.copy()
X_Train = (255*X_Train).astype('uint8')
X_Val = (255*X_Val).astype('uint8')

print('X_Train shape before reshaping = ' + str(X_Train.shape))
print('X_Val shape before reshaping = ' + str(X_Val.shape),"\n")
X_Train = X_Train.reshape(X_Train.shape[0],img_x*img_y*n_channels)
X_Val = X_Val.reshape(X_Val.shape[0],img_x*img_y*n_channels)
print('X_Train shape after reshaping = ' + str(X_Train.shape))
print('X_Val shape after reshaping = ' + str(X_Val.shape))

# Save the reshaped images into pickel file
X_TrainCopy = X_Train.copy()

train_images = []
for i in range(len(X_TrainCopy)):
    train_images.append(X_TrainCopy[i])
# print(len(train_images))  

train_classes = []
for i in range(len(y_train)):
    train_classes.append(y_train[i])
# print(len(train_classes))  


X_Train shape before reshaping = (748, 192, 192, 3)
X_Val shape before reshaping = (731, 192, 192, 3) 

X_Train shape after reshaping = (748, 110592)
X_Val shape after reshaping = (731, 110592)


## Saving Train data to pickle files

In [9]:

X_Train_df = pd.DataFrame()
X_Train_df['Image_Array'] = train_images
X_Train_df['Class'] = train_classes

# Reshuffle the images & it's respective class labels
X_Train_df = X_Train_df.sample(frac=1).reset_index(drop=True)  

X_Train_df.to_pickle('X_Train_'+str(n_classes)+'classes_'+str(img_x)+'_'+str(img_y)+'.pkl') #Save the file as pickle
print('Saved the training images into pickle file')
print(X_Train_df.shape)
X_Train_df.head()

Saved the training images into pickle file
(748, 2)


,Image_Array,Class
0,"[166, 194, 235, 167, 196, 235, 167, 195, 235, ...","[0, 0, 1]"
1,"[53, 52, 48, 74, 75, 68, 76, 79, 69, 80, 82, 7...","[1, 0, 0]"
2,"[38, 30, 24, 46, 33, 31, 47, 39, 36, 54, 42, 3...","[1, 0, 0]"
3,"[255, 255, 255, 255, 255, 255, 255, 255, 254, ...","[1, 0, 0]"
4,"[249, 216, 247, 251, 201, 236, 252, 219, 246, ...","[0, 1, 0]"


## Split the val set into Validation and test by Reshuffling


In [10]:
X_ValCopy = X_Val.copy()
val_images = []
for i in range(len(X_ValCopy)):
    val_images.append(X_ValCopy[i])
# print(len(val_images))  

val_classes = []
for i in range(len(y_val)):
    val_classes.append(y_val[i])
# print(len(val_classes))  

X_Val_df = pd.DataFrame()
X_Val_df['Image_Array'] = val_images
X_Val_df['Class'] = val_classes

print('Original shape of validation_df: ' + str(X_Val_df.shape))

#Reshuffle the images & it's respective class labels
X_Val_df = X_Val_df.sample(frac=1).reset_index(drop=True) 

#Hard coded to 300
X_Test_df = X_Val_df[:300]
X_Val_df = X_Val_df[300:]

X_Val_df.to_pickle('X_Val_'+str(n_classes)+'classes_'+str(img_x)+'_'+str(img_y)+'.pkl') #Save the file as pickle
print('Saved the validation and test images into pickle')
print(X_Val_df.shape)
X_Val_df.head()


Original shape of validation_df: (731, 2)
Saved the validation and test images into pickle
(431, 2)


,Image_Array,Class
300,"[75, 102, 62, 100, 119, 76, 99, 119, 71, 97, 1...","[1, 0, 0]"
301,"[97, 98, 80, 99, 98, 83, 101, 99, 86, 102, 100...","[1, 0, 0]"
302,"[41, 10, 4, 39, 15, 7, 74, 54, 40, 75, 56, 39,...","[1, 0, 0]"
303,"[178, 202, 236, 176, 202, 236, 177, 202, 236, ...","[0, 0, 1]"
304,"[112, 141, 177, 113, 142, 177, 114, 143, 179, ...","[0, 0, 1]"


In [11]:
#Save also test image
X_Test_df.to_pickle('X_Test_'+str(n_classes)+'classes_'+str(img_x)+'_'+str(img_y)+'.pkl') #Save the file as pickle
print('Saved the test images into pickle')
print(X_Test_df.shape)
X_Test_df.head()

Saved the test images into pickle
(300, 2)


,Image_Array,Class
0,"[156, 163, 169, 161, 168, 174, 170, 177, 183, ...","[1, 0, 0]"
1,"[207, 206, 211, 207, 206, 208, 208, 206, 211, ...","[0, 0, 1]"
2,"[62, 67, 57, 66, 70, 59, 65, 72, 66, 71, 76, 6...","[0, 1, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0]"
4,"[88, 104, 118, 95, 113, 126, 89, 107, 117, 94,...","[0, 0, 1]"


## Read the saved images - Only needed to run in Google colab


In [9]:

# train_df = pd.read_pickle('X_Train_2classes_256_256.pkl')
# print('Training images')
# x_train_array = np.array(train_df['Image_Array'].values.tolist())
# x_train = x_train_array.reshape(x_train_array.shape[0],256,256,3)
# print(x_train.shape)
# y_train = np.array(train_df['Class'].values.tolist())
# print(y_train.shape)

# print(train_df.shape)
# train_df.head()

# val_df = pd.read_pickle('X_Val_2classes_256_256.pkl')
# print('validation images')
# x_val_array = np.array(val_df['Image_Array'].values.tolist())
# x_val = x_val_array.reshape(x_val_array.shape[0],256,256,3)
# print(x_val.shape)
# y_val = np.array(val_df['Class'].values.tolist())
# print(y_val.shape)

# print(val_df.shape)
# val_df.head()

# test_df = pd.read_pickle('X_Test_2classes_256_256.pkl')
# test_df.reset_index(drop=True,inplace=True)
# print('testing images')
# x_test_array = np.array(test_df['Image_Array'].values.tolist())
# x_test = x_test_array.reshape(x_test_array.shape[0],256,256,3)
# print(x_test.shape)
# y_test = np.array(test_df['Class'].values.tolist())
# print(y_test.shape)

# print(test_df.shape)
# test_df.head()